In [38]:
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [ ]:
# Set up Chrome options
service  = Service('/Users/Omar/Documents/Python_projects/webscrapping/chromedriver')
driver = webdriver.Chrome(service=service)

In [ ]:
# Navigate to the Dubizzle properties page and start connection
driver.get("https://www.dubizzle.com.eg/en/properties/")
driver.maximize_window()

In [ ]:
all_data = []  # list to store each card's data

cards = driver.find_elements(By.XPATH, "//div[@class='_637fa00f']")

for card in cards:

    # LOCATION
    try:
        location = card.find_element(By.XPATH, ".//span[@class='f7d5e47e']").text
    except:
        location = None

    # SHORT DESCRIPTION
    try:
        short_description = card.find_element(By.XPATH, ".//h2[@class='_562a2db2']").text
    except:
        short_description = None

    # PRICE
    try:
        price = card.find_element(By.XPATH, ".//span[@class='ddc1b288']").text
    except:
        price = None

    # BEDS / BATHS / AREA
    beds = baths = sqm = None
    try:
        info_spans = card.find_elements(By.XPATH, ".//span[@class='_18b01e88']")
        for span in info_spans:
            label = span.get_attribute("aria-label")  # Beds / Bathrooms / Area
            value_el = span.find_element(By.XPATH, ".//span[@class='_3e1113f0']")
            value = value_el.text.strip()

            if label == "Beds":
                beds = value
            elif label == "Bathrooms":
                baths = value
            elif label == "Area":
                sqm = value
    except:
        pass

    # SELLER NAME & PHONE
    seller_name = phone = None
    try:
        # Open listing detail page
        url = card.find_element(By.XPATH, ".//a").get_attribute("href")
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[-1])
        driver.get(url)
        time.sleep(1.0)

        # Try to click Call button to get phone
        try:
            call_btn = driver.find_element(By.XPATH, "//div[@aria-label='Call']//button | //button[.//span[contains(text(),'Call')]]")
            call_btn.click()
            time.sleep(1.0)
            phone_el = driver.find_element(By.XPATH, "//span[@aria-label='Listing phone number']")
            phone = phone_el.text.strip()
        except:
            # fallback regex from page source
            html = driver.page_source
            phones = re.findall(r"(?:\+?2)?01[0-2,5]\d{8}", html)
            phone = phones[0] if phones else None

        # Seller name
        try:
            seller_el = driver.find_element(By.XPATH, "//span[contains(@class,'_3681e431')]")
            seller_name = seller_el.text.strip()
        except:
            seller_name = None

        # Close detail tab
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    except:
        seller_name = phone = None

    # STORE RESULT
    all_data.append({
        "location": location,
        "short_description": short_description,
        "price": price,
        "beds": beds,
        "baths": baths,
        "sqm": sqm,
        "seller_name": seller_name,
        "phone": phone
    })

# CREATE DATAFRAME & SAVE CSV
df = pd.DataFrame(all_data)
df.to_csv("dubizzle_listings.csv", index=False, encoding="utf-8")
print("Saved to dubizzle_listings.csv")

Saved to dubizzle_listings.csv
